In [66]:
import os
import cv2
import keras
import math
import random
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import backend as K
from keras.applications.mobilenetv2 import MobileNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input, InceptionResNetV2
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from skimage import exposure
from skimage.measure import compare_ssim as ssim

In [67]:
# Memory error, so to feed model in batches
batch_size = 20
train_dir = 'C:/Users/n3rDx/Desktop/Homework Upload/Capstone/images/train/'

In [68]:
#custom image augmentation function
def custom_preprocessing(image):
    state = random.randint(0,2)
    if state == 0:
        processed_img = exposure.equalize_adapthist((image*1.0/255), clip_limit=0.02)
    elif state == 1:
        processed_img = exposure.equalize_hist(image)
    elif state == 2:
        p2, p98 = np.percentile(image, (2,98))
        processed_img = exposure.rescale_intensity(image, in_range=(p2,p98))
    return processed_img

image_gen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        vertical_flip=True)
        #preprocessing_function=custom_preprocessing)

In [69]:
#Load weights
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/inception_weights.h5')
inception.graph = tf.get_default_graph()

In [70]:
# function to split training set X train, y train and produce augmented images
def image_a_b_gen(batch_size):
    for img in image_gen.flow_from_directory(train_dir, batch_size=batch_size, class_mode=None,shuffle=True):
        grayscaled_rgb = gray2rgb(rgb2gray(img))
        embed = second_input(grayscaled_rgb)
        lab_batch = rgb2lab(img)
        X_train = lab_batch[:,:,:,0]
        X_train = X_train.reshape(X_train.shape+(1,))
        y_train = lab_batch[:,:,:,1:]/128
        yield ([X_train, second_input(grayscaled_rgb)], y_train)

        
#Create input from inception model
def second_input(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [43]:
for i in image_gen.flow_from_directory(train_dir, batch_size=batch_size, class_mode=None,shuffle=True):
    i = i*1.0/255
    grayscaled_rgb = gray2rgb(rgb2gray(i))
    embed = second_input(grayscaled_rgb)
    lab_batch = rgb2lab(i)
    X_train = lab_batch[:,:,:,0]
    X_train = X_train.reshape(X_train.shape+(1,))
    y_train = lab_batch[:,:,:,1:]
    break

Found 2000 images belonging to 1 classes.


In [44]:
y_train.shape

(20, 256, 256, 2)

In [71]:
class DSSIMObjective:
    def __init__(self, k1=0.01, k2=0.03, max_value=1.0):
        self.__name__ = 'DSSIMObjective'
        self.k1 = k1
        self.k2 = k2
        self.max_value = max_value
        self.backend = K.backend()

    def __int_shape(self, x):
        return K.int_shape(x) if self.backend == 'tensorflow' else K.shape(x)

    def __call__(self, y_true, y_pred):
        ch = K.shape(y_pred)[-1]

        def _fspecial_gauss(size, sigma):
            #Function to mimic the 'fspecial' gaussian MATLAB function.
            coords = np.arange(0, size, dtype=K.floatx())
            coords -= (size - 1 ) / 2.0
            g = coords**2
            g *= ( -0.5 / (sigma**2) )
            g = np.reshape (g, (1,-1)) + np.reshape(g, (-1,1) )
            g = K.constant ( np.reshape (g, (1,-1)) )
            g = K.softmax(g)
            g = K.reshape (g, (size, size, 1, 1)) 
            g = K.tile (g, (1,1,ch,1))
            return g
                  
        kernel = _fspecial_gauss(11,1.5)

        def reducer(x):
            return K.depthwise_conv2d(x, kernel, strides=(1, 1), padding='valid')

        c1 = (self.k1 * self.max_value) ** 2
        c2 = (self.k2 * self.max_value) ** 2
        
        mean0 = reducer(y_true)
        mean1 = reducer(y_pred)
        num0 = mean0 * mean1 * 2.0
        den0 = K.square(mean0) + K.square(mean1)
        luminance = (num0 + c1) / (den0 + c1)
        
        num1 = reducer(y_true * y_pred) * 2.0
        den1 = reducer(K.square(y_true) + K.square(y_pred))
        c2 *= 1.0 #compensation factor
        cs = (num1 - num0 + c2) / (den1 - den0 + c2)

        ssim_val = K.mean(luminance * cs, axis=(-3, -2) )
        return K.mean( (1.0 - ssim_val ) / 2.0 )

In [72]:
ssim_loss = DSSIMObjective()

In [73]:
#Output from inception resnet
inception_input = Input(shape=(1000,))
#Encoder
encoder_input = Input(shape=(256, 256, 1))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)    # BS, 128, 128, 64
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)             # BS, 128, 128, 128
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)  # BS, 64, 64, 128
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)             # BS, 64, 64, 256
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)  # BS, 32, 32, 256
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)             # BS, 32, 32, 512
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)             # BS, 32, 32, 512
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)             # BS, 32, 32, 256
#Fusion
fusion_output = RepeatVector(32 * 32)(inception_input)                                             # BS, 32, 32, 1000
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)                                           # BS, 32, 32, 1000
fusion_output = concatenate([encoder_output, fusion_output], axis=3)                               # BS, 64, 64, 1256
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)              # BS, 64, 64, 256
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)              # BS, 64, 64, 128
decoder_output = Dropout(0.5)(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)                                              # BS, 128, 128, 128
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)              # BS, 128, 128, 64
decoder_output = Dropout(0.5)(decoder_output)     
decoder_output = UpSampling2D((2, 2))(decoder_output) 
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)              # BS, 128, 128, 32
decoder_output = Dropout(0.5)(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)              # BS, 128, 128, 16
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)              # BS, 128, 128, 2
decoder_output = UpSampling2D((2, 2))(decoder_output)                                              # BS, 256, 256, 2
model = Model(inputs=[encoder_input, inception_input], outputs=decoder_output)

#Train model
model.compile(optimizer='adam',loss=ssim_loss ,metrics=['mse','mean_absolute_error'])

In [74]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_995 (Conv2D)             (None, 128, 128, 64) 640         input_32[0][0]                   
__________________________________________________________________________________________________
conv2d_996 (Conv2D)             (None, 128, 128, 128 73856       conv2d_995[0][0]                 
__________________________________________________________________________________________________
conv2d_997 (Conv2D)             (None, 64, 64, 128)  147584      conv2d_996[0][0]                 
__________________________________________________________________________________________________
conv2d_998

In [75]:
model.fit_generator(image_a_b_gen(batch_size), epochs=1, steps_per_epoch=100)

Epoch 1/1
Found 2000 images belonging to 1 classes.
100/100 [==============================] - 3169s 32s/step - loss: 0.4995 - mean_squared_error: 75.6911 - mean_absolute_error: 5.8122


In [24]:
model.save('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/Final_final_inception_ssimloss_dropout.h5')

In [76]:
# Load black and white images
test = []
for filename in os.listdir('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/images/test/test'):
    test.append(img_to_array(load_img('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/images/test/test/'+filename)))
test = np.array(test, dtype=float)
test = 1.0/255*test
test = gray2rgb(rgb2gray(test))
inception_test = second_input(test)
test_input = rgb2lab(test)[:,:,:,0]
test_input = test_input.reshape(test_input.shape+(1,))

In [77]:
# from gray import, get the AB predictions
output = model.predict([test_input,inception_test])
output = output * 128

In [78]:
# Output colorizations
for i in range(len(output)):
    canvas = np.zeros((256, 256, 3))
    canvas[:,:,0] = test[i][:,:,0]
    canvas[:,:,1:] = output[i]
    picture = lab2rgb(canvas)
    imsave("C:/Users/n3rDx/Desktop/Homework Upload/Capstone/results/"+str(i)+".jpg", picture)

C:\Users\n3rDx\Anaconda3\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 524 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
W0703 11:48:19.961088  8584 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\n3rDx\Anaconda3\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 520 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
W0703 11:48:20.016127  8584 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\n3rDx\Anaconda3\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 516 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
W0703 11:48:20.071166  8584 util.py:64] Lossy conversion from float

In [33]:
# Memory error, so to feed model in batches
# Image transformer
batch_size = 20

image_gen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# function to split training set X train, y train and produce augmented images
train_dir = '../Capstone/few_images/'
def image_a_b_gen(batch_size):
    for i in image_gen.flow_from_directory(train_dir, batch_size=batch_size,class_mode=None,shuffle=True):
        i = 1.0/255*i
        lab_batch = rgb2lab(i)
        X_train = lab_batch[:,:,:,0]
        X_train = X_train.reshape(X_train.shape+(1,))
        y_train = lab_batch[:,:,:,1:]/128
        yield ([X_train, y_train])

test_gen = ImageDataGenerator()

In [31]:
#Shared models


#Model A
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output_shared)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
#Model B

#Fusion 
fusion_output = concatenate([encoder_output, global_encoder], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=encoder_input, outputs=decoder_output)
# Finish model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train model
model.fit_generator(image_a_b_gen(batch_size=batch_size), steps_per_epoch=10, epochs=150)